In [7]:
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, ZoomControl, Marker, Popup
from ipywidgets import Output, HBox, VBox, HTML, Text
import bqplot.pyplot as plt
from bqplot import DateScale, LinearScale,OrdinalScale, Bars, Axis, Figure,CATEGORY10
from geopy.geocoders import Nominatim
import asyncio
from ipyflex import FlexLayout 
import json 
import os
import configparser
import random
import requests
import Functions as fct
import ipywidgets as widgets 
config = configparser.ConfigParser() 
config.read('.config')
token = config['DEFAULT']['token']


In [8]:
if not os.path.exists('countries.geojson'):
    url = 'https://github.com/datasets/geo-countries/blob/master/data/countries.geojson'
    r = requests.get(url)
    with open('countries_copy', 'w') as f:
        f.write(r.content.decode("utf-8"))
   
with open('countries.geojson', 'r') as f:
    data = json.load(f)
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }
geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
m = Map(center=(50.6252978589571, 0.34580993652344), zoom=2)

geo_json_layer = GeoJSON(data=geo_json.data)
m.add_layer(geo_json_layer)


In [9]:
box = VBox()
axes_options_1 = {
    "x": {"label": "Time","label_style": {"font-size": "3px"}},
    "y": {"label": "Power Consumption (MWh)", "label_offset": "6ex","label_font_size": "3px"},
}
axes_options_2 = {
    "x": {"label": "Time","label_offset": "6ex"},
    "y": {"label": "Power Value", "label_offset": "6ex"},
}
axes_options_3 = {
    "x": {"label": "Source Energie","label_offset": "6ex"},
    "y": {"label": "Value (MWh)", "label_offset": "6ex"},
}
axes_options_4 = {
    "x": {"label": "Country","label_offset": "6ex"},
    "y": {"label": "Value (MWh)", "label_offset": "6ex"},
}


In [10]:
#Hourly_Power_Consumption_fig
x_ord1 = DateScale()
y_sc1 = LinearScale()

bar = Bars(scales={'x': x_ord1, 'y': y_sc1}, colors=['blue'], padding=0.2)

ax_x = Axis(scale=x_ord1, **axes_options_1["x"])
ax_y = Axis(scale=y_sc1, orientation='vertical', tick_format='0.1e', **axes_options_1["y"])

Hourly_Power_Consumption_fig = Figure(marks=[bar], axes=[ax_x, ax_y], 
             animation_duration=1000, legend_style=dict(fill="var(--jp-layout-color0)"),
             legend_location="bottom-left")
Hourly_Power_Consumption_fig.axes[0].tick_style = {'font-size': '10px'}  
Hourly_Power_Consumption_fig.axes[1].tick_style = {'font-size': '10px'} 
Hourly_Power_Consumption_fig.layout.width = '50%'  
axes = plt.axes(options=axes_options_1)

#Import Export Chart 

x_scale = DateScale()
y_scale = LinearScale()

Exp_bar = Bars(scales={"x": x_scale, "y": y_scale}, padding=0.2,  colors=['blue'])
Imp_bar = Bars(scales={"x": x_scale, "y": y_scale}, padding=0.2, colors=['red'])

ax_x = Axis(scale=x_scale,**axes_options_2["x"])
ax_y = Axis(scale=y_scale, orientation="vertical", tick_format="0.2f",**axes_options_2["y"])

Exp_fig = Figure(axes=[ax_x, ax_y], animation_duration=1000)
Imp_fig=Figure(axes=[ax_x, ax_y], animation_duration=1000)

Exp_fig.marks = [Exp_bar]
Imp_fig.marks = [Imp_bar]

fig_exp_imp = widgets.HBox([Exp_fig, Imp_fig])

Exp_fig.axes[0].tick_style = {'font-size': '10px'}  
Exp_fig.axes[1].tick_style = {'font-size': '10px'}  
Imp_fig.axes[0].tick_style = {'font-size': '10px'}
Imp_fig.axes[1].tick_style = {'font-size': '10px'}

fig_exp_imp.layout.width = '50%'  

# Power by source chart 
x_scale = OrdinalScale()
y_scale = LinearScale()

bar_S = Bars(scales={"x": x_scale, "y": y_scale}, padding=0.2, colors=CATEGORY10, color_mode="group")

ax_x = Axis(scale=x_scale, **axes_options_3["x"])
ax_y = Axis(scale=y_scale, orientation="vertical", tick_format="0.2f", **axes_options_3["y"])

bar_S.orientation = "horizontal"
ax_x.orientation = "vertical"
ax_y.orientation = "horizontal"

source_fig = Figure(axes=[ax_x, ax_y], animation_duration=1000, 
                    axes_options=axes_options_3)
source_fig.axes[0].tick_style = {'font-size': '6px', 'font-weight': 'bold'}  # Pour l'axe x
source_fig.axes[1].tick_style = {'font-size': '10px'}  # Pour l'axe x

source_fig.marks = [bar_S]
#Power imported by country
x_ord3 = OrdinalScale()
y_sc3 = LinearScale()

bar_C = Bars(scales={"x": x_ord3, "y": y_sc3}, padding=0.1, colors=CATEGORY10, color_mode="group")
ax_x = Axis(scale=x_ord3,**axes_options_4["x"])
ax_y = Axis(scale=y_sc3, orientation="vertical", tick_format="0.2f",**axes_options_4["y"])

source_fig_c = Figure(axes=[ax_x, ax_y], animation_duration=1000, 
                    axes_options=axes_options_3)
source_fig_c.axes[0].tick_style = {'font-size': '10px'}  
source_fig_c.axes[1].tick_style = {'font-size': '10px'}  # Pour l'axe x

source_fig_c.marks = [bar_C]

def handle_click(event, feature, **kwargs):
    country = feature['properties']['ADMIN']
    
    Hourly_Power_Consumption_fig.title="Hourly Power Consumption {}".format(country)
    fig_exp_imp.title="Hourly Power Consumption {}".format(country)
    Exp_fig.title="Power Export of {}".format(country)
    Imp_fig.title="Power Import of {}".format(country)
    source_fig.title="Power by Source {}".format(country)
    source_fig_c.title="Power Import by County {}".format(country)
    
    task= asyncio.create_task(fct.Hourly_Power_Consumption(country, bar, token))
    task1= asyncio.create_task(fct.Hourly_Imp_Exp_Power_Consumption(country, Imp_bar, Exp_bar, token))
    task2=asyncio.create_task(fct.Power_Consumption_Source(country,bar_S,token))
    task3=asyncio.create_task(fct.Power_Imported_Country(country,bar_C,token))


#  Electricity Map Dashboard



You can get your token at this link: https://api-portal.electricitymaps.com/catalog/prd_8e3ood60rcv8f51h

In [11]:
text_widget = widgets.Text(
    placeholder='Please enter your Token',
    description='Token:',
    disabled=False
)

def handle_text_change(change):
    global token
    token = change.new
    
text_widget.observe(handle_text_change, names='value')
text_widget

Text(value='', description='Token:', placeholder='Please enter your Token')

In [12]:
geo_json_layer.on_click(handle_click)

charts_hbox = HBox([Hourly_Power_Consumption_fig,fig_exp_imp])
box.children = [m, charts_hbox,source_fig,source_fig_c] 

box